### GASS evolution

nb to test the evolution functions 

In [1]:
using PyCall
using DataFrames
using Printf

#rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

coord= pyimport("astropy.coordinates")
am= pyimport("astropy.modeling.models")
mf= pyimport("astropy.modeling.fitting")

np= pyimport("numpy")


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [2]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    param::Array{Dict{String,Float64},2}    ## beam and mrs for each subarray    
end

In [3]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res , [b , mrs])
end

_fitness_subarray (generic function with 1 method)

In [4]:
## Creation of a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
            println(res)
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(0, pop, fitness, score , paramsub)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [5]:
## get best parents
##

function _get_elitism(cfg, pop::_population)
    isort= reverse(sortperm(pop.score))
    
    elit= []
    for i in 1:cfg.ga.Number_Elitism
        push!(elit, [pop.subarr[isort[i],:], pop.fitness[isort[i],:], pop.score[isort[i]]])
    end
    return(elit)
end

## using tournament selection
function _get_parents(cfg, pop::_population)
    tour= cfg.ga.Tournament_Size
    
    popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
    isort= reverse(sortperm(pop.score[popran[1:tour]]))
    iparent1= popran[isort[1]]
  
    iparent2= iparent1
    while iparent1==iparent2
        popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
        isort= reverse(sortperm(pop.score[popran[1:tour]]))
        iparent2= popran[isort[1]] 
    end 
    return(iparent1 , iparent2)
end

_get_parents (generic function with 1 method)

In [6]:
function _flatten_subarray(cfg, subarr)
    subflat= zeros(Int, cfg.obs.Antenna_Number)
    
    counter= 1
    for i in 1:cfg.obs.Subarray_Number
        for j in 1:cfg.sub.Pads_Per_Subarray[i]
            subflat[counter]= subarr[i][j]
            counter += 1
        end
    end
    return(subflat)
end

## reform the subarray array..
function _wrap_subarray(cfg, sub)
    subarray= Array{Array{Int,1}}(undef,0)

    for i in 1:cfg.obs.Subarray_Number
        push!(subarray, sub[cfg.sub.Subrange[i]])
    end
    
    return(subarray)
end

## crossover using the two parents
##
function _get_crossover1(cfg, pop::_population , parents)
    nmax= cfg.obs.Antenna_Number
    p1= _flatten_subarray(cfg, pop.subarr[parents[1],:])
    p2= _flatten_subarray(cfg, pop.subarr[parents[2],:])
    child= zeros(Int, nmax)

    pivot= rand(1:nmax,2)
    pivot1= minimum(pivot) ; pivot2= maximum(pivot)
    child[pivot1:pivot2] .= p1[pivot1:pivot2]

    irange1= 1:pivot1-1 ; irange2= pivot2+1:nmax
    
    cnterp2= 1
    for i in irange1
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end
    for i in irange2
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end  
    
    # println(child)
    c= _wrap_subarray(cfg, child)
    # println(c)
    return(child)
end

_get_crossover1 (generic function with 1 method)

In [7]:
## mutation
## loop over all alleles and swap two if p is true
## 

function _get_mutation(cfg, child)
    nmax= cfg.obs.Antenna_Number
    si= copy(child)
    
    pmutation= 1-cfg.ga.Mutation_Rate
    rng = Random.MersenneTwister()
    rd= Random.rand(rng, nmax)
    # println(rd)
    for i in 1:nmax
        if rd[i] > pmutation
            iswap= Random.rand(rng, big.(1:nmax))
            value= si[iswap]
            si[iswap]= si[i]
            si[i]= value
        end
    end
    return(si)
end

_get_mutation (generic function with 1 method)

In [8]:
## Evolve one population
##

function _get_evolution(cfg, pi::_population)
    npop= cfg.ga.Population_Size
    age= pi.age + 1
    nelit= cfg.ga.Number_Elitism
    
    ## Elitism
    pelit=  _get_elitism(cfg, pi)
    
    ## parent selection, crossover and mutation
    ncross= npop-nelit
    
    crosspop= []
    for i in 1:ncross
        parent= _get_parents(cfg, pi)
        child=  _get_crossover1(cfg, pi , parent)
        mutated= _get_mutation(cfg, child)
        push!(crosspop, mutated)
        println(parent)
        println(child)
    end

    
    ## create an evolved population
    pinew= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    for i in 1:nelit
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pelit[i][1][j]
            fitness[i,j], res= _fitness_subarray(cfg, j, pinew[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end    
    
    println("fitness of crossover..")
    for i in nelit+1:npop
        println(crosspop[i-nelit])
        pwrap= _wrap_subarray(cfg, crosspop[i-nelit])
        println(pwrap)
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pwrap[j]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pinew[i,j])
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(age, pinew, fitness, score, paramsub)
    
    return(_pop)
end

_get_evolution (generic function with 1 method)

In [9]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Creating the population...")
    p0= _create_population(cfg)
    
    println("Elitism..")
    pelit=  _get_elitism(cfg, p0)
    
    println("Crossover...")
    parent= _get_parents(cfg, p0)
    child=  _get_crossover1(cfg, p0 , parent)
    println(child)
    
    println("Mutatis mutandis...")
    mutated= _get_mutation(cfg, child)
    
    ## evolution
    println("Evolution...")
    
    species= []
    for i in 1:cfg.ga.Number_Iterations
        #if (i % 10) == 0
            @printf("\n## Iteration: %d \n",i)
        #end
        
        p1= _get_evolution(cfg, p0)
        push!(species,p1)
        p0=p1
    end
    
    ## PLOT ####
    #PyPlot.imshow(h)
    #PyPlot.colorbar()
    #PyPlot.show()
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Creating the population...
1 1 2.3386413924164806
Any[synthbeam(2.46799, 2.39525, 2.43135, 1.03037, 6.28472), 6.89187]
1 2 -1.1023383084454639
Any[synthbeam(4.86073, 3.

9 1 2.334572236100486
9 2 0.4597507397282161
9 3 -11.00301456316822
9 4 -13.51035278372328
[12, 8, 27, 38, 36, 49, 35, 10, 24, 26, 44, 43, 11, 42, 41, 5, 28, 31, 6, 16, 17, 32, 45, 29, 3, 4, 22, 9, 13, 23, 20, 2, 40, 18, 34, 1, 37, 15, 47, 50, 48, 46, 30, 25, 39, 33, 19, 14, 21, 7]
Array{Int64,1}[[12, 8, 27, 38, 36, 49, 35, 10, 24, 26, 44, 43, 11, 42, 41, 5, 28, 31, 6, 16, 17, 32, 45, 29, 3, 4, 22, 9, 13, 23], [20, 2, 40, 18, 34, 1, 37, 15, 47, 50], [48, 46, 30, 25, 39, 33, 19], [14, 21, 7]]
10 1 0.18603416330057754
10 2 1.503096139411462
10 3 -11.10794028059312
10 4 -10.557477838337045

## Iteration: 2 
(7, 2)
[21, 38, 6, 12, 8, 27, 35, 10, 24, 26, 44, 43, 11, 42, 36, 49, 41, 5, 28, 31, 16, 17, 32, 45, 29, 20, 4, 22, 9, 13, 23, 3, 2, 40, 18, 34, 1, 37, 15, 47, 50, 48, 46, 30, 25, 39, 33, 19, 14, 7]
(7, 1)
[28, 25, 18, 38, 36, 49, 7, 48, 40, 42, 32, 47, 27, 44, 6, 4, 22, 23, 31, 34, 11, 29, 41, 13, 19, 37, 3, 50, 24, 43, 21, 30, 46, 1, 2, 14, 16, 15, 35, 9, 45, 33, 17, 26, 8, 39, 20, 1

10 1 1.0248645470342157
10 2 -4.563781874104743
10 3 -11.476835993694685
10 4 3.539169280459884

## Iteration: 4 
(9, 1)
[38, 23, 36, 28, 18, 12, 7, 4, 40, 46, 32, 47, 27, 44, 6, 26, 39, 45, 31, 34, 11, 29, 41, 13, 19, 37, 3, 50, 24, 43, 21, 30, 42, 49, 2, 5, 17, 1, 14, 16, 15, 35, 22, 9, 33, 8, 20, 25, 10, 48]
(9, 1)
[21, 38, 6, 23, 31, 36, 50, 27, 24, 37, 32, 28, 18, 42, 12, 7, 11, 41, 13, 34, 4, 40, 19, 43, 3, 47, 29, 22, 30, 44, 45, 46, 26, 49, 2, 5, 17, 1, 14, 16, 15, 35, 9, 33, 8, 39, 20, 25, 10, 48]
(1, 9)
[38, 23, 36, 12, 7, 4, 40, 46, 24, 37, 32, 28, 18, 47, 27, 44, 6, 26, 39, 45, 31, 34, 11, 29, 41, 13, 19, 3, 50, 43, 21, 30, 42, 49, 2, 5, 17, 1, 14, 16, 15, 35, 9, 33, 8, 22, 20, 25, 10, 48]
(8, 9)
[38, 36, 28, 12, 7, 4, 40, 18, 46, 32, 47, 27, 6, 26, 39, 45, 31, 34, 11, 29, 41, 13, 19, 37, 3, 50, 24, 43, 21, 30, 49, 42, 1, 2, 44, 14, 16, 15, 35, 9, 23, 5, 17, 33, 8, 22, 20, 25, 10, 48]
(8, 1)
[38, 6, 31, 36, 27, 32, 28, 18, 7, 11, 41, 13, 34, 40, 19, 49, 47, 29, 22, 45, 46, 